<a href="https://colab.research.google.com/github/HananeNourMoussa/darija-ner/blob/master/Experiment_1_BERT_Dense_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Experiment Parameters:

In [ ]:
# UBC-NLP/ARBERT
# UBC-NLP/MARBERT
# bert-base-multilingual-cased
# SI2M-Lab/DarijaBERT
# aubmindlab/bert-base-arabertv02
# CAMeL-Lab/bert-base-arabic-camelbert-da
# CAMeL-Lab/bert-base-arabic-camelbert-msa
# CAMeL-Lab/bert-base-arabic-camelbert-mix

In [ ]:
config_default = {
    'epochs' : 5,
    'batch_size': 8,
    'lr_bert': 3e-5,
    'lr_rest':1e-4,
    'hidden_size': 256,
    'num_layers': 1,
    'eps': 1e-8,
    "lm": "aubmindlab/bert-base-arabertv02",
    "p1":0.3,
    "p2": 0.2,
    "dataset": "DarNERcorp-data-in-brief"
}

In [ ]:
MAX_LEN = 100
lm = "aubmindlab/bert-base-arabertv02"
batch_size = 8
freeze_bert = False

In [ ]:
#GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 29 15:47:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# Installations and Imports:

In [ ]:
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install torch -q
!pip install wandb -Uq
#!pip install camel-tools --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 120.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import transformers
from transformers import BertForTokenClassification, AutoTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
from google.colab import drive
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
import numpy as np
from tqdm import tqdm, trange
from statistics import mean
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import LongTensor
import wandb
import os
import random
#from camel_tools.ner import NERecognizer
#from camel_tools.tokenizers.word import simple_word_tokenize

In [ ]:
#ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def get_metrics(true, pred):
  acc = accuracy_score(true, pred)
  f1 = f1_score(true, pred)
  prec = precision_score(true, pred)
  recall = recall_score(true, pred)
  report = classification_report(true, pred, digits = 6)
  return acc, f1, prec, recall, report

# Baseline Performance:

### On DarNERcorp_test:

In [ ]:
ner = NERecognizer('CAMeL-Lab/bert-base-arabic-camelbert-mix-ner')

In [ ]:
darner_test = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/DarNERcorp_test.csv")

Mounted at /content/gdrive/


In [ ]:
#darner_test_sentence = ' '.join(darner_test["Token"])
darner_test_sentence = list(darner_test['Token'])

In [ ]:
darner_test_tags = list(darner_test['Tag'])

In [ ]:
predictions = ner.predict_sentence(darner_test_sentence)

In [ ]:
predictions = ['B-PER' if item == 'B-PERS' else item for item in predictions]
predictions = ['I-PER' if item == 'I-PERS' else item for item in predictions]

In [ ]:
darner_test_tags = [darner_test_tags]
darner_predictions = [predictions]

In [ ]:
#get metrics
dar_acc, dar_f1, dar_prec, dar_recall, dar_report = get_metrics(darner_test_tags, darner_predictions)

In [ ]:
print(dar_acc,'\n', dar_f1,'\n', dar_prec,'\n', dar_recall,'\n', dar_report,'\n')

0.8853566009104704 
 0.44745484400656815 
 0.47063903281519864 
 0.42644757433489827 
               precision    recall  f1-score   support

         LOC       0.52      0.68      0.59       539
        MISC       0.26      0.05      0.08       395
         ORG       0.47      0.29      0.36       190
         PER       0.39      0.68      0.49       154

   micro avg       0.47      0.43      0.45      1278
   macro avg       0.41      0.43      0.38      1278
weighted avg       0.42      0.43      0.39      1278
 



### On mixedNERcorp_test:

In [ ]:
mixedner_test = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/MixedNERcorp_test.csv")
mixedner_test_sentence =  list(mixedner_test['Token'])
mixedner_test_tags = [list(mixedner_test['Tag'])]


In [ ]:
len(mixedner_test_sentence)

38188

In [ ]:
#mixed_sentence = simple_word_tokenize(mixedner_test_sentence)
predictions = ner.predict_sentence(mixedner_test_sentence)
predictions = ['B-PER' if item == 'B-PERS' else item for item in predictions]
predictions = ['I-PER' if item == 'I-PERS' else item for item in predictions]

In [ ]:
len(predictions)

38188

In [ ]:
mixedner_predictions = [predictions]

In [ ]:
#get metrics
mixed_acc, mixed_f1, mixed_prec, mixed_recall, mixed_report = get_metrics(mixedner_test_tags, mixedner_predictions)
print(mixed_acc,'\n',mixed_f1,'\n',mixed_prec,'\n',mixed_recall,'\n',mixed_report)

0.9407405467686184 
 0.680728113262063 
 0.7013992259601072 
 0.6612405276452428 
               precision    recall  f1-score   support

         LOC       0.70      0.82      0.75      1215
        MISC       0.58      0.25      0.35       638
         ORG       0.68      0.55      0.61       649
         PER       0.74      0.80      0.77      1061

   micro avg       0.70      0.66      0.68      3563
   macro avg       0.68      0.60      0.62      3563
weighted avg       0.69      0.66      0.66      3563



# Data Preprocessing:

In [ ]:
drive.mount('/content/gdrive/', force_remount=True)
dataset = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/MixedNERcorp_train.csv")
dataset.pop("Unnamed: 0")
dataset

Mounted at /content/gdrive/


,Sentence,Token,Tag
0,0,Uppsala,B-LOC
1,0,),O
2,0,هيّا,O
3,0,رابع,O
4,0,أكبر,O
...,...,...,...
177821,5957,في,O
177822,5957,مختلف,O
177823,5957,أنحاء,O
177824,5957,المصنع,O


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(lm)

In [ ]:
class SentenceGetter(object):
  def __init__(self, data):
    #start from the first sentence
    self.n_sent = 1
    self.data = data
    self.empty = False
    #aggregate token and tag
    agg_func = lambda s:[(to, ta) for to, ta in zip (s["Token"].values.tolist(),
                                                     s["Tag"].values.tolist())]
    self.grouped = self.data.groupby("Sentence").apply(agg_func)
    #make list of sentences
    self.sentences = [s for s in self.grouped]
  def get_next(self):
    try:
      #get current sentence
      s = self.grouped["Sentence: {}".format(self.n_sent)]
      #move to next sentence
      self.n_sent += 1
      return s
    except:
      return None

In [ ]:
getter = SentenceGetter(dataset)

In [ ]:
#get tokens of sentences
sentences = [[token[0] for token in sentence] for sentence in getter.sentences]
sentences[0]

['Uppsala',
 ')',
 'هيّا',
 'رابع',
 'أكبر',
 'مدينة',
 'ف',
 'سّويد',
 'من',
 'بعد',
 'سطوكهولم',
 '،',
 'ݣوتنبورݣ',
 'ؤ',
 'مالمو',
 '.']

In [ ]:
#remove a nan value that causes errors later
sentences_ = list()
for sentence in sentences:
  sentence_ = list(filter(lambda item: type(item) == str, sentence))
  sentences_.append(sentence_)

In [ ]:
#getting tags
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
print(labels[0])

['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-LOC', 'O']


In [ ]:
#adding PAD to tags and assigning label numbers
tag_values = list(set(dataset["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
#function to tokenize each word into subwords while preserving corresponding labels
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
#pass each sentence at a time to the function
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences_, labels)
]

In [ ]:
#separate text and labeLs
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
#convert text to input ids
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
#getting encoded tags with PAD included for CLS and SEP tokens
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
#setting attention masks to tell model to ignore PADs
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
#split data (sentences) to train, validation, and test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.20)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.20)

In [ ]:
#convert to pyTorch tensors
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

<ipython-input-34-183560a6cb13>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tr_inputs = torch.tensor(tr_inputs)
<ipython-input-34-183560a6cb13>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tr_tags = torch.tensor(tr_tags)
<ipython-input-34-183560a6cb13>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tr_masks = torch.tensor(tr_masks)


In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

# Model Definitions:

#### For Token Classification Code:

In [ ]:
def evaluate_model(model, dataset, total_loss = 0):
  model.eval()
  predictions = []
  truth = []
  with torch.no_grad():
      for batch in dataset:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the acuracy for this batch of test sentences.
        total_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        truth.extend(label_ids)
  avg_loss = total_loss / len(dataset)
  pred_tags = [tag_values[p_i] for p, l in zip(predictions, truth)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
  true_tags = [tag_values[l_i] for l in truth
                                  for l_i in l if tag_values[l_i] != "PAD"]
  pred = list()
  pred.append(pred_tags)
  true = list()
  true.append(true_tags)
  return pred, true, avg_loss

In [ ]:
model = BertForTokenClassification.from_pretrained(
        lm,
        num_labels=len(tag2idx),
        output_attentions = False,
        output_hidden_states = False,
        hidden_dropout_prob=0.3)
model.cuda()
optimizer = optim.AdamW([
    {'params': model.bert.parameters(), 'lr':3e-5},
    {'params': model.classifier.parameters(), 'lr':1e-4}
])
total_steps = len(train_dataloader) * 5
scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

In [ ]:
training_loss = []
validation_loss = []
tr_accuracies, val_accuracies = [], []
tr_f1s, val_f1s = [], []
tr_precisions, val_precisions = [], []
tr_recalls, val_recalls = [], []
criterion = nn.CrossEntropyLoss().to(device)
def train(config = None):
  with wandb.init(project = 'final-results', config=config_default):
    config = wandb.config
    # Total number of training steps is number of batches * number of epochs.
    #config = transformers.BertConfig.from_pretrained(pretrained_model_name_or_path = config.lm, hidden_dropout_prob=config.p1)


    # Create the learning rate scheduler.
    # Train the model
    for epoch in trange(config.epochs, desc = "Epoch"):
        # Training phase
        model.train()
        total_loss = 0
        for step, batch in enumerate(train_dataloader):
            # add batch to gpu
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            # Always clear any previously calculated gradients before performing a backward pass.
            model.zero_grad()
            # forward pass
            # This will return the loss (rather than the model output)
            # because we have provided the `labels`.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
            # get the loss
            loss = outputs[0]
            # Perform a backward pass to calculate the gradients.
            loss.backward()
            # track train loss
            total_loss += loss.item()
            # Clip the norm of the gradient
            # This is to help prevent the "exploding gradients" problem.
            #torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            # update parameters
            optimizer.step()
            # Update the learning rate.
            scheduler.step()
        avg_train_loss = total_loss / len(train_dataloader)
        print("Average train loss: {}".format(avg_train_loss))
        training_loss.append(avg_train_loss)
        #log train loss
        wandb.log({'train_loss': avg_train_loss, "epoch": epoch})
        #performance on training
        tr_pred, tr_true, _ = evaluate_model(model, train_dataloader)
        tr_acc, tr_f1, tr_prec, tr_recall, tr_report = get_metrics(tr_true, tr_pred)
        print("Training Accuracy: {}".format(tr_acc))
        tr_accuracies.append(tr_acc)
        wandb.log({'train_acc': tr_acc, "epoch": epoch})
        print("Training F1-Score: {}".format(tr_f1))
        tr_f1s.append(tr_f1)
        wandb.log({'train_f1': tr_f1, "epoch": epoch})
        print("Training Precision: {}".format(tr_prec))
        tr_precisions.append(tr_prec)
        wandb.log({'train_prec': tr_prec, "epoch": epoch})
        print("Training Recall: {}".format(tr_recall))
        tr_recalls.append(tr_recall)
        wandb.log({'train_recall': tr_recall, "epoch": epoch})
        print("Training Classification Report:\n {}".format(tr_report))
        print()
  #model.save(os.path.join(wandb.run.dir, "CAMeL_msa_ex_1.h5"))
train()
#wandb.agent(sweep_id, train)

wandb: Currently logged in as: hanane-n-moussa (darijaner). Use `wandb login --relogin` to force relogin


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Average train loss: 0.17319461998167204


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch:  20%|██        | 1/5 [02:14<08:58, 134.59s/it]

Training Accuracy: 0.9462158717709916
Training F1-Score: 0.6929164504411002
Training Precision: 0.6834293026231606
Training Recall: 0.7026707012235233
Training Classification Report:
               precision    recall  f1-score   support

          AD   0.000000  0.000000  0.000000         0
         LOC   0.715972  0.838839  0.772550      6509
        MISC   0.538222  0.426057  0.475616      3239
         ORG   0.575895  0.567031  0.571429      2014
         PER   0.797872  0.784884  0.791325      3440

   micro avg   0.683429  0.702671  0.692916     15202
   macro avg   0.525592  0.523362  0.522184     15202
weighted avg   0.678075  0.702671  0.686888     15202


Average train loss: 0.07489792498503656


...

Epoch:  80%|████████  | 4/5 [09:04<02:16, 136.34s/it]

Training Accuracy: 0.9669930395568844
Training F1-Score: 0.8188057652711049
Training Precision: 0.7787206266318538
Training Recall: 0.8632416787264834
Training Classification Report:
               precision    recall  f1-score   support

          AD   0.000000  0.000000  0.000000         0
         LOC   0.816327  0.921801  0.865863      6509
        MISC   0.647899  0.728311  0.685756      3239
         ORG   0.727597  0.810328  0.766737      2014
         PER   0.870484  0.910465  0.890026      3440

   micro avg   0.778721  0.863242  0.818806     15202
   macro avg   0.612461  0.674181  0.641676     15202
weighted avg   0.780941  0.863242  0.819824     15202


Average train loss: 0.043154689067687164


Epoch: 100%|██████████| 5/5 [11:20<00:00, 136.19s/it]

Training Accuracy: 0.9724707122199893
Training F1-Score: 0.8417316792428698
Training Precision: 0.8189000870971755
Training Recall: 0.8658729114590186
Training Classification Report:
               precision    recall  f1-score   support

          AD   0.000000  0.000000  0.000000         0
         LOC   0.864458  0.918113  0.890478      6509
        MISC   0.700291  0.743748  0.721365      3239
         ORG   0.755484  0.820755  0.786768      2014
         PER   0.889553  0.908430  0.898893      3440

   micro avg   0.818900  0.865873  0.841732     15202
   macro avg   0.641957  0.678209  0.659501     15202
weighted avg   0.820722  0.865873  0.842611     15202




epoch,▁▁▁▁▁▃▃▃▃▃▅▅▅▅▅▆▆▆▆▆█████
train_acc,▁▄▇▇█
train_f1,▁▄▇▇█
train_loss,█▃▂▁▁
train_prec,▁▄▇▆█
train_recall,▁▅▇██
epoch,4
train_acc,0.97247
train_f1,0.84173
train_loss,0.04315
train_prec,0.8189


In [ ]:
training_loss = []
validation_loss = []
tr_accuracies, val_accuracies = [], []
tr_f1s, val_f1s = [], []
tr_precisions, val_precisions = [], []
tr_recalls, val_recalls = [], []
criterion = nn.CrossEntropyLoss().to(device)
def train(config = None):
  with wandb.init(project = 'final-results', config=config_default):
    config = wandb.config
    # Total number of training steps is number of batches * number of epochs.
    #config = transformers.BertConfig.from_pretrained(pretrained_model_name_or_path = config.lm, hidden_dropout_prob=config.p1)


    # Create the learning rate scheduler.
    # Train the model
    for epoch in trange(config.epochs, desc = "Epoch"):
        # Training phase
        model.train()
        total_loss = 0
        for step, batch in enumerate(train_dataloader):
            # add batch to gpu
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            # Always clear any previously calculated gradients before performing a backward pass.
            model.zero_grad()
            # forward pass
            # This will return the loss (rather than the model output)
            # because we have provided the `labels`.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
            # get the loss
            loss = outputs[0]
            # Perform a backward pass to calculate the gradients.
            loss.backward()
            # track train loss
            total_loss += loss.item()
            # Clip the norm of the gradient
            # This is to help prevent the "exploding gradients" problem.
            #torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            # update parameters
            optimizer.step()
            # Update the learning rate.
            scheduler.step()
        avg_train_loss = total_loss / len(train_dataloader)
        print("Average train loss: {}".format(avg_train_loss))
        training_loss.append(avg_train_loss)
        #log train loss
        wandb.log({'train_loss': avg_train_loss, "epoch": epoch})
        #performance on training
        tr_pred, tr_true, _ = evaluate_model(model, train_dataloader)
        tr_acc, tr_f1, tr_prec, tr_recall, tr_report = get_metrics(tr_true, tr_pred)
        print("Training Accuracy: {}".format(tr_acc))
        tr_accuracies.append(tr_acc)
        wandb.log({'train_acc': tr_acc, "epoch": epoch})
        print("Training F1-Score: {}".format(tr_f1))
        tr_f1s.append(tr_f1)
        wandb.log({'train_f1': tr_f1, "epoch": epoch})
        print("Training Precision: {}".format(tr_prec))
        tr_precisions.append(tr_prec)
        wandb.log({'train_prec': tr_prec, "epoch": epoch})
        print("Training Recall: {}".format(tr_recall))
        tr_recalls.append(tr_recall)
        wandb.log({'train_recall': tr_recall, "epoch": epoch})
        print("Training Classification Report:\n {}".format(tr_report))
        print()
        # performance on validation
        val_pred, val_true, avg_val_loss = evaluate_model(model, valid_dataloader)
        validation_loss.append(avg_val_loss)
        wandb.log({'validation_loss': avg_val_loss, "epoch": epoch})
        val_acc, val_f1, val_prec, val_recall, val_report = get_metrics(val_true, val_pred)
        print("Validation Accuracy: {}".format(val_acc))
        val_accuracies.append(val_acc)
        wandb.log({'val_acc': val_acc, "epoch": epoch})
        print("Validation F1-Score: {}".format(val_f1))
        val_f1s.append(val_f1)
        wandb.log({'val_f1': val_f1, "epoch": epoch})
        print("Validation Precision: {}".format(val_prec))
        val_precisions.append(val_prec)
        wandb.log({'val_prec': val_prec, "epoch": epoch})
        print("Validation Recall: {}".format(val_recall))
        val_recalls.append(val_recall)
        wandb.log({'val_recall': val_recall, "epoch": epoch})
        overfitting = abs(tr_f1-val_f1)
        wandb.log({'overfitting': overfitting, "epoch": epoch})
        print("Validation Classification Report:\n {}".format(val_report))
        print()
  #model.save(os.path.join(wandb.run.dir, "CAMeL_msa_ex_1.h5"))
train()
#wandb.agent(sweep_id, train)

wandb: Currently logged in as: hanane-n-moussa (darijaner). Use `wandb login --relogin` to force relogin


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Average train loss: 0.322092370958484


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training Accuracy: 0.8664169109928693
Training F1-Score: 0.4515728155339806
Training Precision: 0.4608433734939759
Training Recall: 0.44266788487894015
Training Classification Report:
               precision    recall  f1-score   support

          AD       0.00      0.00      0.00         0
         LOC       0.51      0.68      0.58      2878
        MISC       0.41      0.38      0.39      2477
         ORG       0.17      0.00      0.01       540
         PER       0.10      0.02      0.03       672

   micro avg       0.46      0.44      0.45      6567
   macro avg       0.24      0.22      0.20      6567
weighted avg       0.40      0.44      0.41      6567




...

Epoch:  80%|████████  | 4/5 [03:07<00:47, 47.02s/it]

Validation Accuracy: 0.9098464049862729
Validation F1-Score: 0.6322055137844611
Validation Precision: 0.6002379535990482
Validation Recall: 0.6677696889477167
Validation Classification Report:
               precision    recall  f1-score   support

          AD       0.00      0.00      0.00         0
         LOC       0.68      0.77      0.72       774
        MISC       0.57      0.64      0.60       466
         ORG       0.29      0.43      0.35        95
         PER       0.60      0.43      0.50       176

   micro avg       0.60      0.67      0.63      1511
   macro avg       0.43      0.45      0.43      1511
weighted avg       0.61      0.67      0.63      1511


Average train loss: 0.10472857982070002
Training Accuracy: 0.9275346581242987
Training F1-Score: 0.6731763354394027
Training Precision: 0.6368699904904225
Training Recall: 0.7138723922643521
Training Classification Report:
               precision    recall  f1-score   support

          AD       0.00      0.00    

Epoch: 100%|██████████| 5/5 [03:55<00:00, 47.09s/it]

Validation Accuracy: 0.9114046152704608
Validation F1-Score: 0.6404945904173106
Validation Precision: 0.6009280742459396
Validation Recall: 0.685638649900728
Validation Classification Report:
               precision    recall  f1-score   support

          AD       0.00      0.00      0.00         0
         LOC       0.68      0.77      0.72       774
        MISC       0.55      0.67      0.61       466
         ORG       0.31      0.44      0.37        95
         PER       0.61      0.49      0.54       176

   micro avg       0.60      0.69      0.64      1511
   macro avg       0.43      0.47      0.45      1511
weighted avg       0.61      0.69      0.64      1511




epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
overfitting,▃▁█▇█
train_acc,▁▅▇██
train_f1,▁▅▇██
train_loss,█▃▂▁▁
train_prec,▁▅▆██
train_recall,▁▄▇██
val_acc,▁▆▇██
val_f1,▁▅▇██
val_prec,▁▆▆██
val_recall,▁▄▇▇█


# Evaluation on Test Set:

### On DarNERcorp_test:

In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/Capstone/DarijaBERT_EX1')

In [ ]:
#sentence from wikipedia darija (not in dataset)
darner_test = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/DarNERcorp_test.csv")
test_darner_set = ' '.join(darner_test["Token"])
tokenized_sentence = tokenizer.encode(test_darner_set)

Token indices sequence length is longer than the specified maximum sequence length for this model (17957 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
new_labels = []
new_tokens = []
n = len(tokenized_sentence)//512
for i in range(0, n+1):
  if (i == n):
    batch = tokenized_sentence[i*512:len(tokenized_sentence)]
  else:
    batch = tokenized_sentence[i*512:i*512+512]
  input_ids = torch.tensor([batch]).cuda()
  with torch.no_grad():
      output = model(input_ids)
  label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
  # join bpe split tokens
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          #adding subword to previous word
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(tag_values[label_idx])
          new_tokens.append(token)

In [ ]:
dar_predictions = [new_labels[1:len(new_labels)-1]]
dar_tags = [list(darner_test['Tag'])]

In [ ]:
dar_acc, dar_f1, dar_prec, dar_recall, dar_report = get_metrics(dar_tags, dar_predictions)
print(dar_acc,'\n', dar_prec,'\n', dar_recall,'\n', dar_f1,'\n', dar_report)

0.9348254931714719 
 0.6154366960054164 
 0.7112676056338029 
 0.65989110707804 
               precision    recall  f1-score   support

          AD   0.000000  0.000000  0.000000         0
         LOC   0.669903  0.896104  0.766667       539
        MISC   0.552707  0.491139  0.520107       395
         ORG   0.525822  0.589474  0.555831       190
         PER   0.727273  0.779221  0.752351       154

   micro avg   0.615437  0.711268  0.659891      1278
   macro avg   0.495141  0.551188  0.518991      1278
weighted avg   0.619173  0.711268  0.657391      1278



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### On mixedNERcorp_test:

In [ ]:
import unicodedata as ud
def remove_punc(s):
  if (len(s) > 1) and s!= '--':
    s = ''.join(c for c in s if not ud.category(c).startswith('P'))
  return s

In [ ]:
mixner_test = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/MixedNERcorp_test.csv")

In [ ]:
tokens = list(mixner_test["Token"])
tokens = list(map(remove_punc, tokens))

In [ ]:
test_mixner_set = ' '.join(tokens)
tokenized_sentence = tokenizer.encode(test_mixner_set)

In [ ]:
new_labels = []
new_tokens = []
n = len(tokenized_sentence)//512
for i in range(0, n+1):
  if (i == n):
    batch = tokenized_sentence[i*512:len(tokenized_sentence)]
  else:
    batch = tokenized_sentence[i*512:i*512+512]
  input_ids = torch.tensor([batch]).cuda()
  with torch.no_grad():
      output = model(input_ids)
  label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
  # join bpe split tokens
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          #adding subword to previous word
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(tag_values[label_idx])
          new_tokens.append(token)

In [ ]:
mix_predictions = [new_labels[1:len(new_labels)-1]]
mix_tags = [list(mixner_test['Tag'])]

In [ ]:
mix_acc, mix_f1, mix_prec, mix_recall, mix_report = get_metrics(mix_tags, mix_predictions)
print(mix_acc,'\n', mix_prec,'\n', mix_recall,'\n', mix_f1,'\n', mix_report)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.9434115428930554 
 0.6423020883924235 
 0.7423519506034241 
 0.6887124072386407 
               precision    recall  f1-score   support

          AD   0.000000  0.000000  0.000000         0
         LOC   0.730954  0.876543  0.797156      1215
        MISC   0.541133  0.463950  0.499578       638
         ORG   0.622614  0.653313  0.637594       649
         PER   0.839844  0.810556  0.824940      1061

   micro avg   0.642302  0.742352  0.688712      3563
   macro avg   0.546909  0.560872  0.551854      3563
weighted avg   0.709656  0.742352  0.723080      3563



## Share model on HuggingFace:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
repo_name = "finetuned-arabert-darija-ner"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/hananour/finetuned-arabert-darija-ner/commit/5ccec3473843080500517dfd5884a454797e4e04', commit_message='Upload tokenizer', commit_description='', oid='5ccec3473843080500517dfd5884a454797e4e04', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
label_names = list(set(dataset["Tag"].values))
label_names.append("PAD")

In [ ]:
label_names

['B-MISC',
 'B-ORG',
 'I-MISC',
 'I-ORG',
 'O',
 'B-PER',
 'B-LOC',
 'I-PER',
 'I-LOC',
 'PAD']

In [ ]:
model.config.id2label = {str(i): lbl for i, lbl in enumerate(label_names)}
model.config.label2id = {lbl: str(i) for i, lbl in enumerate(label_names)}

In [ ]:
repo_name = "finetuned-arabert-darija-ner"
model.config.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/hananour/finetuned-arabert-darija-ner/commit/e961e2b92b75e9740b69c157968ab52101d7ba18', commit_message='Upload config', commit_description='', oid='e961e2b92b75e9740b69c157968ab52101d7ba18', pr_url=None, pr_revision=None, pr_num=None)